In [ ]:
from __future__ import unicode_literals, print_function
import spacy
import en_core_web_sm
import random
from pathlib import Path
from spacy.util import minibatch, compounding
import tarfile
import xml.etree.ElementTree as ET
import os
from pandas import ExcelWriter
import pandas as pd
from sklearn.model_selection import train_test_split
import timeit
# from spacy.gold import GoldParse
from spacy.scorer import Scorer

In [ ]:
%pip install spacy


In [ ]:
# function to combine all the available and save to a new excel file
def xml2Excel(inDirectory = None, outFileName = None, outDirectory = None):
    path = inDirectory
    rawTexts = []
    tags = []
    for filename in os.listdir(path):
        if not filename.endswith('.xml'): continue
        fullname = os.path.join(path, filename)
        tree = ET.parse(fullname)
        root = tree.getroot()
        rawText = root[0].text
        tag = []
        for child in root[1]:
            tag.append(child.attrib)
        rawTexts.append(rawText)
        tags.append(tag)
    fullPath = outDirectory + outFileName
    writer = ExcelWriter(outFileName)
    df = pd.DataFrame({'text': rawTexts, 'tags' : tags})
    df.to_excel(writer,'Sheet1', index = False)
    writer.close()
xml2Excel(inDirectory = '/content/drive/MyDrive/i2b2_2014_deidentification/combined_training/',
         outFileName = 'PythonExport2014_train.xlsx',
         outDirectory = './')

In [ ]:
xml2Excel(inDirectory = '/content/drive/MyDrive/i2b2_2014_deidentification/testing-PHI-Gold-fixed/',
         outFileName = 'PythonExport2014_test.xlsx',
         outDirectory = './')

In [ ]:
dfTrain = pd.read_excel('./PythonExport2014_train.xlsx')

In [ ]:
dfTest = pd.read_excel('./PythonExport2014_test.xlsx')

In [ ]:
dfTest

,text,tags
0,\n\n\nRecord date: 2073-03-25\n\n\nTHE SPECIAL...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
1,\n\n\nRecord date: 2095-05-10\n\n\n\n\n\n\n\nN...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
2,\n\n\nRecord date: 2145-07-25\n\n\n\nOlga come...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
3,\n\n\nRecord date: 2066-01-13\n\n\n\nPatient i...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
4,\n\n\nRecord date: 2091-09-18\n\n\n\nReason fo...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
...,...,...
509,\n\n\nRecord date: 2059-06-19\n\nPROMPTCARE IN...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
510,\n\n\nRecord date: 2096-12-13\n\n\n\n\n\n\n\nT...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
511,\n\n\nRecord date: 2091-07-11\n\nReason for Vi...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
512,\n\n\nRecord date: 2093-04-11\n\nMEDIQUIK Neur...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."


In [ ]:
dfTrain

,text,tags
0,\n\n\nRecord date: 2137-06-01\n\n\n\n\n\n\n\n\...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
1,\n\n\nRecord date: 2071-02-03\n\n ...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
2,\n\n\nRecord date: 2092-02-13\n\n\n\nProblems\...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
3,\n\n\nRecord date: 2087-09-18\n\n\n\n\n\n\n\n\...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
4,\n\n\nRecord date: 2097-06-28\n\n\n\nVisit Not...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
...,...,...
785,\n\n\nRecord date: 2091-03-19\n\n\n\nCCU Admis...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
786,\n\n\nRecord date: 2133-11-13\n\nRheumatology ...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
787,"\n\n\nRecord date: 2074-02-22\n\n \nGregg, Llo...","[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
788,\n\n\nRecord date: 2062-01-29\n\nTRIBAL INTERN...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."


In [ ]:
def count_words(text):
    return len(text.split())
dfTrain['word_count'] = dfTrain['text'].apply(count_words)

In [ ]:
dfTrain['word_count'].describe()

count     790.000000
mean      618.867089
std       358.606872
min        74.000000
25%       359.000000
50%       524.500000
75%       823.750000
max      2984.000000
Name: word_count, dtype: float64

In [ ]:
dfTest['word_count'] = dfTest['text'].apply(count_words)

In [ ]:
dfTest['word_count'].describe()

count     514.000000
mean      615.198444
std       345.948648
min        99.000000
25%       361.000000
50%       534.000000
75%       804.250000
max      2474.000000
Name: word_count, dtype: float64

In [ ]:
dfTest

text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  \n\n\nRecord date: 2073-03-25\n\n\nTHE SPECIALTY HOSPITAL OF AUSTIN\n\nCLINIC NOTE \n\nVictor Lujan\nSMH#:  018-92-33\nMarch 25, 2073\n\n\nDIAGNOSIS:\n\nBilateral hand numbness and pain, recurrent carpal tunnel syndrome.\n\nSUBJECTIVE:\n\nThe patient is a 54-year-old right-hand dominant male with multiple medical problems including diabetes mellitus and r

In [ ]:
type(dfTest['text'])

pandas.core.series.Series

In [ ]:
type(dfTest['tags'])

pandas.core.series.Series

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
def func(txt, tgt):
  words = []
  ctag  = []
  j = 0
  for i in tgt:
    k = int(i['start'])
    l = int(i['end'])
    for tt in txt[j:k].strip().split(" "):
      tt = tt.replace("\n", '')
      tt = tt.replace("\n", '').strip()
      if tt != '':
        words.append(tt)
        ctag.append("O")
    info = i["text"].split(" ")
    ctag.append(f"B-{i['TYPE']}")
    for _ in info[1:]:
      ctag.append(f"I-{i['TYPE']}")
    words += [ii.strip() for ii in info]
    j = l
  for tt in txt[j:len(txt)].strip().split(" "):
    tt = tt.replace("\n", '')
    tt = tt.replace("\n", '').strip()
    if tt != '':
      words.append(tt)
      ctag.append("O")
  return words, ctag

In [ ]:
from tqdm import tqdm
import json
import ast
from tqdm import tqdm

In [ ]:
df_train = {'Doc #' : [], 'Words' : [], 'Tags' : [],'segment #' : []}

for i in tqdm(range(dfTrain.shape[0])):
  txt = dfTrain .iloc[i]['text']
  tgt = ast.literal_eval(dfTrain .iloc[i]['tags'])
  a,b = func(txt, tgt)
  df_train["Words"] += a
  df_train["Tags"] += b
  df_train['Doc #'] += [f"Doc-{i}"]* len(a)
  x = int(len(a)/100)
  y = len(a)%100
  for j in range(x):
    df_train['segment #'] += [f"segment {i} {j}"]*100
  df_train['segment #'] += [f"segment {i} {j+1}"]*y
df_train = pd.DataFrame(df_train)

100%|██████████| 790/790 [00:01<00:00, 706.98it/s]


In [ ]:
df_test = {'Doc #' : [], 'Words' : [], 'Tags' : [],'segment #' : []}

for i in tqdm(range(dfTest.shape[0])):
  txt = dfTest .iloc[i]['text']
  tgt = ast.literal_eval(dfTest .iloc[i]['tags'])
  a,b = func(txt, tgt)
  df_test["Words"] += a
  df_test["Tags"] += b
  df_test['Doc #'] += [f"Doc-{i}"]* len(a)
  x = int(len(a)/100)
  y = len(a)%100
  for j in range(x):
    df_test['segment #'] += [f"segment {i} {j}"]*100
  df_test['segment #'] += [f"segment {i} {j+1}"]*y
df_test = pd.DataFrame(df_test)

100%|██████████| 514/514 [00:00<00:00, 791.43it/s]


In [ ]:
# Remove symbols at the start or end of the 'Tokens' column
df_train['Words'] = df_train['Words'].str.replace(r'^[\/;,.:\[\]*]+|[\/;,.:\[\]*]+$', '')

<ipython-input-74-f38a096c8243>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_train['Words'] = df_train['Words'].str.replace(r'^[\/;,.:\[\]*]+|[\/;,.:\[\]*]+$', '')


In [ ]:
df_test

,Doc #,Words,Tags,segment #
0,Doc-0,Record,O,segment 0 0
1,Doc-0,date:,O,segment 0 0
2,Doc-0,2073-03-25,B-DATE,segment 0 0
3,Doc-0,THE,B-HOSPITAL,segment 0 0
4,Doc-0,SPECIALTY,I-HOSPITAL,segment 0 0
...,...,...,...,...
302456,Doc-513,IN,B-DOCTOR,segment 513 3
302457,Doc-513,/,O,segment 513 3
302458,Doc-513,rowe,B-DOCTOR,segment 513 3
302459,Doc-513,/,O,segment 513 3


In [ ]:
# Remove symbols at the start or end of the 'Tokens' column
df_test['Words'] = df_test['Words'].str.replace(r'^[\/;,.:\[\]*]+|[\/;,.:\[\]*]+$', '')

<ipython-input-77-798b8633aa4a>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['Words'] = df_test['Words'].str.replace(r'^[\/;,.:\[\]*]+|[\/;,.:\[\]*]+$', '')


In [ ]:
df_test

,Doc #,Words,Tags,segment #
0,Doc-0,Record,O,segment 0 0
1,Doc-0,date,O,segment 0 0
2,Doc-0,2073-03-25,B-DATE,segment 0 0
3,Doc-0,THE,B-HOSPITAL,segment 0 0
4,Doc-0,SPECIALTY,I-HOSPITAL,segment 0 0
...,...,...,...,...
302456,Doc-513,IN,B-DOCTOR,segment 513 3
302457,Doc-513,,O,segment 513 3
302458,Doc-513,rowe,B-DOCTOR,segment 513 3
302459,Doc-513,,O,segment 513 3


In [ ]:
df_train.isnull().sum()

Doc #        0
Words        0
Tags         0
segment #    0
dtype: int64

In [ ]:
df_test['Words'].isnull().sum()

0

In [ ]:
df_train

,Doc #,Words,Tags,segment #
0,Doc-0,Record,O,segment 0 0
1,Doc-0,date,O,segment 0 0
2,Doc-0,2137-06-01,B-DATE,segment 0 0
3,Doc-0,LAW,B-PATIENT,segment 0 0
4,Doc-0,ELLEN,I-PATIENT,segment 0 0
...,...,...,...,...
466422,Doc-789,Drive,I-STREET,segment 789 2
466423,Doc-789,Boulder,B-CITY,segment 789 2
466424,Doc-789,,O,segment 789 2
466425,Doc-789,CO,B-STATE,segment 789 2


In [ ]:
df_train['Tags'].value_counts()

O                   443109
B-DATE                7502
B-DOCTOR              2885
I-DOCTOR              2170
B-HOSPITAL            1437
B-PATIENT             1316
B-AGE                 1233
I-HOSPITAL            1135
I-DATE                 878
I-PATIENT              722
B-MEDICALRECORD        611
I-STREET               445
B-CITY                 394
B-STATE                314
B-PHONE                309
B-USERNAME             264
B-IDNUM                261
B-PROFESSION           234
B-STREET               216
B-ZIP                  212
I-PROFESSION           192
B-ORGANIZATION         124
I-ORGANIZATION         112
I-CITY                  90
B-COUNTRY               66
I-IDNUM                 54
I-PHONE                 52
I-MEDICALRECORD         25
I-AGE                   13
B-FAX                    8
I-COUNTRY                8
I-LOCATION-OTHER         8
B-DEVICE                 7
B-LOCATION-OTHER         4
B-EMAIL                  4
I-URL                    4
I-STATE                  3
B

In [ ]:
df_test['Tags'].value_counts()

O                   287195
B-DATE                4980
B-DOCTOR              1912
I-DOCTOR              1380
B-PATIENT              879
B-HOSPITAL             875
B-AGE                  764
I-HOSPITAL             700
I-DATE                 507
I-PATIENT              479
B-MEDICALRECORD        422
I-STREET               283
B-CITY                 260
B-PHONE                215
B-IDNUM                195
B-STATE                190
B-PROFESSION           179
I-PROFESSION           159
B-ZIP                  140
B-STREET               136
B-COUNTRY              117
B-USERNAME              92
B-ORGANIZATION          82
I-CITY                  81
I-ORGANIZATION          61
I-PHONE                 48
I-IDNUM                 41
I-MEDICALRECORD         26
I-STATE                 15
I-COUNTRY               13
B-LOCATION-OTHER        13
B-DEVICE                 8
I-LOCATION-OTHER         7
I-DEVICE                 2
B-FAX                    2
I-AGE                    1
I-FAX                    1
B

In [ ]:
df_train = df_train.dropna()

In [ ]:
df_test = df_test.dropna()

In [ ]:
df_test.nunique()

Doc #          514
Words        41069
Tags            38
segment #     1776
dtype: int64

In [ ]:
df_test.nunique()

Doc #          514
Words        41069
Tags            38
segment #     1776
dtype: int64

In [ ]:
df_train.nunique()

Doc #          790
Words        55912
Tags            42
segment #     2718
dtype: int64

In [ ]:
df_train.nunique()

Doc #          790
Words        55912
Tags            42
segment #     2718
dtype: int64

In [ ]:
df_train['Tags'].value_counts()

O                   443109
B-DATE                7502
B-DOCTOR              2885
I-DOCTOR              2170
B-HOSPITAL            1437
B-PATIENT             1316
B-AGE                 1233
I-HOSPITAL            1135
I-DATE                 878
I-PATIENT              722
B-MEDICALRECORD        611
I-STREET               445
B-CITY                 394
B-STATE                314
B-PHONE                309
B-USERNAME             264
B-IDNUM                261
B-PROFESSION           234
B-STREET               216
B-ZIP                  212
I-PROFESSION           192
B-ORGANIZATION         124
I-ORGANIZATION         112
I-CITY                  90
B-COUNTRY               66
I-IDNUM                 54
I-PHONE                 52
I-MEDICALRECORD         25
I-AGE                   13
B-FAX                    8
I-COUNTRY                8
I-LOCATION-OTHER         8
B-DEVICE                 7
B-LOCATION-OTHER         4
B-EMAIL                  4
I-URL                    4
I-STATE                  3
B

#Mapping Tag set of n2c2_2014 dataset with n2c2_2006 dataset

In [ ]:
df_train['Tags'][df_train['Tags'] == 'B-EMAIL'] = 'B-ID'
df_train['Tags'][df_train['Tags'] == 'B-URL'] = 'B-ID'
df_train['Tags'][df_train['Tags'] == 'I-FAX'] = 'I-ID'
df_train['Tags'][df_train['Tags'] == 'B-MEDICALRECORD'] = 'B-ID'
df_train['Tags'][df_train['Tags'] == 'I-MEDICALRECORD'] = 'I-ID'
df_train['Tags'][df_train['Tags'] == 'B-USERNAME'] = 'B-ID'
df_train['Tags'][df_train['Tags'] == 'B-IDNUM'] = 'B-ID'
df_train['Tags'][df_train['Tags'] == 'I-IDNUM'] = 'I-ID'
df_train['Tags'][df_train['Tags'] == 'B-BIOID'] = 'B-ID'
df_train['Tags'][df_train['Tags'] == 'B-HEALTHPLAN'] = 'B-ID'
df_train['Tags'][df_train['Tags'] == 'I-HEALTHPLAN'] = 'I-ID'
df_train['Tags'][df_train['Tags'] == 'I-STREET'] = 'I-LOCATION'
df_train['Tags'][df_train['Tags'] == 'B-STREET'] = 'B-LOCATION'
df_train['Tags'][df_train['Tags'] == 'B-CITY'] = 'B-LOCATION'
df_train['Tags'][df_train['Tags'] == 'I-CITY'] = 'I-LOCATION'
df_train['Tags'][df_train['Tags'] == 'B-STATE'] = 'B-LOCATION'
df_train['Tags'][df_train['Tags'] == 'I-STATE'] = 'I-LOCATION'
df_train['Tags'][df_train['Tags'] == 'B-ORGANIZATION'] = 'B-LOCATION'
df_train['Tags'][df_train['Tags'] == 'I-ORGANIZATION'] = 'I-LOCATION'
df_train['Tags'][df_train['Tags'] == 'B-CITY'] = 'B-LOCATION'
df_train['Tags'][df_train['Tags'] == 'I-CITY'] = 'I-LOCATION'
df_train['Tags'][df_train['Tags'] == 'B-COUNTRY'] = 'B-LOCATION'
df_train['Tags'][df_train['Tags'] == 'I-COUNTRY'] = 'I-LOCATION'
df_train['Tags'][df_train['Tags'] == 'B-LOCATION-OTHER'] = 'B-LOCATION'
df_train['Tags'][df_train['Tags'] == 'I-LOCATION-OTHER'] = 'I-LOCATION'
df_train['Tags'][df_train['Tags'] == 'I-AGE'] = 'B-AGE'
df_train['Tags'][df_train['Tags'] == 'B-PROFESSION'] = 'B-PATIENT'
df_train['Tags'][df_train['Tags'] == 'I-PROFESSION'] = 'I-PATIENT'
df_train['Tags'][df_train['Tags'] == 'B-ZIP'] = 'B-LOCATION'
df_train['Tags'][df_train['Tags'] == 'B-DEVICE'] = 'B-PHONE'
df_train['Tags'][df_train['Tags'] == 'B-FAX'] = 'B-PHONE'
df_train['Tags'][df_train['Tags'] == 'I-URL'] = 'I-PHONE'


In [ ]:
df_test['Tags'][df_test['Tags'] == 'B-EMAIL'] = 'B-ID'
df_test['Tags'][df_test['Tags'] == 'B-URL'] = 'B-ID'
df_test['Tags'][df_test['Tags'] == 'I-FAX'] = 'I-ID'
df_test['Tags'][df_test['Tags'] == 'B-MEDICALRECORD'] = 'B-ID'
df_test['Tags'][df_test['Tags'] == 'I-MEDICALRECORD'] = 'I-ID'
df_test['Tags'][df_test['Tags'] == 'B-USERNAME'] = 'B-ID'
df_test['Tags'][df_test['Tags'] == 'B-IDNUM'] = 'B-ID'
df_test['Tags'][df_test['Tags'] == 'I-IDNUM'] = 'I-ID'
df_test['Tags'][df_test['Tags'] == 'B-BIOID'] = 'B-ID'
df_test['Tags'][df_test['Tags'] == 'B-HEALTHPLAN'] = 'B-ID'
df_test['Tags'][df_test['Tags'] == 'I-HEALTHPLAN'] = 'I-ID'
df_test['Tags'][df_test['Tags'] == 'I-STREET'] = 'I-LOCATION'
df_test['Tags'][df_test['Tags'] == 'B-STREET'] = 'B-LOCATION'
df_test['Tags'][df_test['Tags'] == 'B-CITY'] = 'B-LOCATION'
df_test['Tags'][df_test['Tags'] == 'I-CITY'] = 'I-LOCATION'
df_test['Tags'][df_test['Tags'] == 'B-STATE'] = 'B-LOCATION'
df_test['Tags'][df_test['Tags'] == 'I-STATE'] = 'I-LOCATION'
df_test['Tags'][df_test['Tags'] == 'B-ORGANIZATION'] = 'B-LOCATION'
df_test['Tags'][df_test['Tags'] == 'I-ORGANIZATION'] = 'I-LOCATION'
df_test['Tags'][df_test['Tags'] == 'B-CITY'] = 'B-LOCATION'
df_test['Tags'][df_test['Tags'] == 'I-CITY'] = 'I-LOCATION'
df_test['Tags'][df_test['Tags'] == 'B-COUNTRY'] = 'B-LOCATION'
df_test['Tags'][df_test['Tags'] == 'I-COUNTRY'] = 'I-LOCATION'
df_test['Tags'][df_test['Tags'] == 'B-LOCATION-OTHER'] = 'B-LOCATION'
df_test['Tags'][df_test['Tags'] == 'I-LOCATION-OTHER'] = 'I-LOCATION'
df_test['Tags'][df_test['Tags'] == 'I-AGE'] = 'B-AGE'
df_test['Tags'][df_test['Tags'] == 'B-PROFESSION'] = 'B-PATIENT'
df_test['Tags'][df_test['Tags'] == 'I-PROFESSION'] = 'I-PATIENT'
df_test['Tags'][df_test['Tags'] == 'B-ZIP'] = 'B-LOCATION'
df_test['Tags'][df_test['Tags'] == 'B-DEVICE'] = 'B-PHONE'
df_test['Tags'][df_test['Tags'] == 'B-FAX'] = 'B-PHONE'
df_test['Tags'][df_test['Tags'] == 'I-URL'] = 'I-PHONE'
df_test['Tags'][df_test['Tags'] == 'I-DEVICE'] = 'B-PHONE'


In [ ]:
df_train

,Doc #,Words,Tags,segment #
0,Doc-0,Record,O,segment 0 0
1,Doc-0,date,O,segment 0 0
2,Doc-0,2137-06-01,B-DATE,segment 0 0
3,Doc-0,LAW,B-PATIENT,segment 0 0
4,Doc-0,ELLEN,I-PATIENT,segment 0 0
...,...,...,...,...
466422,Doc-789,Drive,I-LOCATION,segment 789 2
466423,Doc-789,Boulder,B-LOCATION,segment 789 2
466424,Doc-789,,O,segment 789 2
466425,Doc-789,CO,B-LOCATION,segment 789 2


In [ ]:
df_train['Tags'].value_counts()

O             443109
B-DATE          7502
B-DOCTOR        2885
I-DOCTOR        2170
B-PATIENT       1550
B-HOSPITAL      1437
B-LOCATION      1330
B-AGE           1246
B-ID            1144
I-HOSPITAL      1135
I-PATIENT        914
I-DATE           878
I-LOCATION       666
B-PHONE          324
I-ID              81
I-PHONE           56
Name: Tags, dtype: int64

In [ ]:
df_test['Tags'].value_counts()

O             287195
B-DATE          4980
B-DOCTOR        1912
I-DOCTOR        1380
B-PATIENT       1058
B-LOCATION       938
B-HOSPITAL       875
B-AGE            765
B-ID             710
I-HOSPITAL       700
I-PATIENT        638
I-DATE           507
I-LOCATION       460
B-PHONE          227
I-ID              68
I-PHONE           48
Name: Tags, dtype: int64

In [ ]:
df_train.to_csv('/content/drive/MyDrive/i2b2_train_2014_to_2006.csv')
df_test.to_csv('/content/drive/MyDrive/i2b2_test_2014_to_2006.csv')
df_test.to_csv('/content/drive/MyDrive/i2b2_val_2014_to_2006.csv')